## Map–Reduce Framework

This notebook shows how Agentics supports **Map–Reduce–style** workloads over typed states, using:

- `Number` → `RomanNumber` transductions (Map / aMap)
- Aggregation over lists of `Number` instances (Reduce / areduce)

We’ll start by defining our core types.

Number is our source type, wrapping an integer.
	•	RomanNumber is our target type, representing its Roman numeral form.
	•	We import agentics.core.transducible_functions so we can use transducible machinery (e.g. <<, With, Transduce, etc.) in the following cells.

In [ ]:
from pydantic import BaseModel
from typing import Optional
import agentics.core.transducible_functions 
class Number(BaseModel):
    number:Optional[int]
class RomanNumber(BaseModel):
    roman_number:Optional[str] =None

🟩 aMap

aMap is the asynchronous “map” layer of Agentics’ transduction engine.

When you create a transducible function (either via @transducible() or using the << operator), it can operate in two modes:
- Single input: X → Y
- List input (aMap): list[X] → list[Y]

When a list is provided, agentics uses an asynchronous map (aMap) to apply the transducible function over the list of inputs.

Below, we dynamically construct a transducible function Number → RomanNumber and apply it to a list of Number objects

In [ ]:
to_roman_number = RomanNumber << Number
roman_numbers = await to_roman_number([Number(number=i) for i in range(1,5)])
for roman in roman_numbers: print(roman)


Agentics lets you configure runtime behavior of a transduction via With(...).
You can control parameters like:
- batch_size – how many items per LLM call / batch.
- timeout – maximum time to wait for a batch.
- persist_output – a file path where outputs are appended in JSONL format.

This is useful when you:
- Run long or large batch jobs,
- Want to checkpoint progress to disk,
- Need reproducibility or auditing of outputs.

Key points:
- The call to to_roman_number([...]) still returns a list[RomanNumber].
- Additionally, every result is appended to /tmp/roman_numbers.jsonl as line-delimited JSON.
- If the file already exists, new results are appended, not overwritten.

In [ ]:
from agentics.core.transducible_functions import With
to_roman_number = RomanNumber << With(Number,
                                      batch_size = 5,
                                      timeout = 20,
                                      persist_output = "/tmp/roman_numbers.jsonl")
roman_numbers = await to_roman_number([Number(number=i) for i in range(1,20)])
print(open("/tmp/roman_numbers.jsonl").read())
pass

🟦 AReduce


A reduce function aggregates multiple SOURCE_TYPE instances into a single TARGET_TYPE instance.

It has the shape: list[SOURCE]  →  TARGET

When such a function returns Transduce(source_list), Agentics interprets it as an areduce request:
- The full list is automatically batched into chunks (e.g. ~10k tokens each),
- Each batch is sent to the LLM asynchronously,
- The LLM returns one TARGET_TYPE per batch,

The snippet below shows a two-step pattern:
- map = Number << RomanNumber — a transduction that (logically) maps Roman numbers back into Number.
- reduce = Number << With(Number, areduce=True, instructions="...") — a configured reduce that asks the LLM to sum the numbers.



In [ ]:
to_number = Number << RomanNumber
reduce = Number << With(Number, 
    areduce=True,
    instructions="return the sum of the input numbers' number fields")

await reduce(await to_number(roman_numbers ))

#### Programmatic vs. llm based areduce

We can also define code-based and LLM-based areduce functions directly via @transducible(areduce=True).

This will behave in the same way as function executed by LLMs. 

However, llm based computation allows us to go a step further and directly sum up roman numbers without writing an otherwise complicated logic. 


In [ ]:
from agentics.core.transducible_functions import transducible, Transduce
@transducible(areduce=True)
async def sum_all_numbers_code(numbers: list[Number]) -> Number :
    sum=0
    for number in numbers:
        sum+=number.number
    return Number(number=sum)


@transducible(areduce=True)
async def sum_all_numbers_llm(numbers: list[Number]) -> Number :
    return Transduce(numbers)

@transducible(areduce=True)
async def sum_all_roman_numbers(numbers: list[RomanNumber]) -> Number :
    return Transduce(numbers)

print(await sum_all_numbers_code([Number(number=92),Number(number=8)]))
print(await sum_all_numbers_llm([Number(number=92),Number(number=8)]))
print(await sum_all_roman_numbers([RomanNumber(roman_number="XCII"),RomanNumber(roman_number="VIII")]))